##### Copyright 2019 The TensorFlow Authors.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Обучение нескольких рабочих процессов с помощью Keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />Смотрите на TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Запустите в Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/ru/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />Изучайте код на GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/ru/tutorials/distribute/multi_worker_with_keras.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Скачайте ноутбук</a>
  </td>
</table>

Note: Вся информация в этом разделе переведена с помощью русскоговорящего Tensorflow сообщества на общественных началах. Поскольку этот перевод не является официальным, мы не гарантируем что он на 100% аккуратен и соответствует [официальной документации на английском языке](https://www.tensorflow.org/?hl=en). Если у вас есть предложение как исправить этот перевод, мы будем очень рады увидеть pull request в [tensorflow/docs](https://github.com/tensorflow/docs) репозиторий GitHub. Если вы хотите помочь сделать документацию по Tensorflow лучше (сделать сам перевод или проверить перевод подготовленный кем-то другим), напишите нам на [docs-ru@tensorflow.org list](https://groups.google.com/a/tensorflow.org/forum/#!forum/docs-ru).

## Введение

В этом руководстве демонстрируется распределенное обучение нескольких воркеров с помощью модели Keras с использованием `tf.distribute.Strategy`, в частности `tf.distribute.experimental.MultiWorkerMirroredStrategy`. С помощью этой стратегии модель Keras, которая была разработана для работы с одним рабочим процессом, может беспрепятственно работать с несколькими воркерами с минимальным изменением кода.

[Распределенное обучение в TensorFlow](../../guide/distribute_training.ipynb) - руководство по распределенным стратегиям, поддерживаемым TensorFlow, для тех, кто заинтересован в более глубоком понимании `tf.distribute.Strategy`.


## Установка и импорт

In [ ]:
import json
import os
import sys

Перед импортом TensorFlow внесите несколько изменений в рабочее окружение.

Отключите все графические процессоры. Это предотвращает ошибки, вызванные тем, что все воркеры пытаются использовать один и тот же графический процессор. В реальном приложении каждый воркер будет работать на разных машинах.

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

Сбросьте переменную окружения TF_CONFIG, о ней мы поговорим позже.

In [ ]:
os.environ.pop('TF_CONFIG', None)

Убедитесь, что текущий каталог находится в путях Python. Это позже позволит ноутбуку импортировать файлы, записанные с помощью `%%writefile`.

In [ ]:
if '.' not in sys.path:
  sys.path.insert(0, '.')

Теперь импортируемt TensorFlow.

In [ ]:
import tensorflow as tf

### Датасет и определение модели

Next create an `mnist.py` file with a simple model and dataset setup. This python file will be used by the worker-processes in this tutorial:

Теперь создайте файл `mnist.py` с простой моделью и установками для датасета. Этот файл будет использоваться рабочими процессами в этом руководстве:

In [ ]:
%%writefile mnist.py

import os
import tensorflow as tf
import numpy as np

def mnist_dataset(batch_size):
  (x_train, y_train), _ = tf.keras.datasets.mnist.load_data()
  # `x` это список типа uint8 и содержит числа в диапазоне [0, 255].
  # вам нужно конвертировать их во float32 и значениями в диапазоне [0, 1]
  x_train = x_train / np.float32(255)
  y_train = y_train.astype(np.int64)
  train_dataset = tf.data.Dataset.from_tensor_slices(
      (x_train, y_train)).shuffle(60000).repeat().batch(batch_size)
  return train_dataset

def build_and_compile_cnn_model():
  model = tf.keras.Sequential([
      tf.keras.Input(shape=(28, 28)),
      tf.keras.layers.Reshape(target_shape=(28, 28, 1)),
      tf.keras.layers.Conv2D(32, 3, activation='relu'),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10)
  ])
  model.compile(
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
      metrics=['accuracy'])
  return model

Попробуйте обучить модель на небольшом количестве эпох и понаблюдайте за результатами одного воркера, чтобы убедиться, что все работает правильно. По мере обучения потери должны уменьшаться, а точность - увеличиваться.

In [ ]:
import mnist

batch_size = 64
single_worker_dataset = mnist.mnist_dataset(batch_size)
single_worker_model = mnist.build_and_compile_cnn_model()
single_worker_model.fit(single_worker_dataset, epochs=3, steps_per_epoch=70)

## Многопользовательская конфигурация

Теперь давайте познакомимся с обучением нескольких воркеров. В TensorFlow переменная среды окружения `TF_CONFIG` требуется для обучения на нескольких машинах, каждая из которых, может иметь свою роль. `TF_CONFIG` - это строка JSON, используемая для указания конфигурации кластера для каждого воркера, являющегося частью кластера.

Вот пример конфигурации:

In [ ]:
tf_config = {
    'cluster': {
        'worker': ['localhost:12345', 'localhost:23456']
    },
    'task': {'type': 'worker', 'index': 0}
}

Вот тот же `TF_CONFIG`, сериализованный как JSON-строка:

In [ ]:
json.dumps(tf_config)

`TF_CONFIG` состоит из двух компонентов: `cluster` и `task`.

* `cluster` одинаков для всех воркеров и содержит информацию об обучающем кластере вцелом, это словарь, состоящий из разных типов заданий, например `worker`. При обучении нескольких воркеров с помощью `MultiWorkerMirroredStrategy` обычно определяется один `worker`, который берет на себя немного больше ответственности, например, сохранение контрольной точки и написание сводного файла для TensorBoard в дополнение к тому, что делает обычный `worker`. Такого воркера называют `главным`, и обычно `worker` с индексом 0 назначается главным (именно так реализуется `tf.distribute.Strategy`).

* `task` содержит информацию о текущей задаче и отличается для каждого воркера. Он определяет `type` и `index` этого воркера.

В этом примере вы устанавливаете `type` задачи в значение` worker` и `index` задачи в значение `0`. Эта машина является первым воркером, который будет главным в кластере и будет выполнять больше работы, чем другие. Обратите внимание, что на других машинах также должна быть установлена переменная среды `TF_CONFIG`, и у нее должен быть такой же словарь с ключем `cluster`, но с другим типом задачи или индекса, в зависимости от роли этих машин.

В качестве примера в этом руководстве показано, как можно установить `TF_CONFIG` с двумя рабочими процессами на `localhost`. На практике пользователи могут создать несколько рабочих процессов на внешних IP-адресах/портах и сконфигурировать `TF_CONFIG` соответственно для каждого воркера.

В этом примере вы будете использовать 2 воркера, `TF_CONFIG` первого воркера показан выше. Для второго воркера вы должны установить `tf_config ['task'] ['index'] = 1`

В данном случае `tf_config` - это просто локальная переменная в Python. Чтобы действительно использовать его для настройки обучения, этот словарь нужно сериализовать в JSON и записать в переменную окружения `TF_CONFIG`.

### Переменные окружения и подпроцессы в ноутбуках

Подпроцессы наследуют переменные окружения от своего родителя. Таким образом, если вы установите переменную окружения `jupyter notebook` в этом процессе:

In [ ]:
os.environ['GREETINGS'] = 'Hello TensorFlow!'

Вы можете получить доступ к этой переменной из подпроцессов:

In [ ]:
%%bash
echo ${GREETINGS}

В следующем разделе вы будете использовать переменные окружения, чтобы передать `TF_CONFIG` рабочим подпроцессам. Вы никогда не запустите свои рабочие процессы таким образом в реальном окружении, но этого достаточно для данного руководства: продемонстрировать минимальный пример с несколькими воркерам.

## Выбор правильной стратегии

В TensorFlow есть две основные формы распределенного обучения:

* Синхронное обучение, при котором шаги обучения синхронизируются между воркерами и репликами, а также
* Асинхронное обучение, при котором шаги обучения не синхронизированы.

В этом руководстве будет показана стратегия `tf.distribute.experimental.MultiWorkerMirroredStrategy`, рекомендуемая для синхронного обучения нескольких воркеров.

`MultiWorkerMirroredStrategy` создает копии всех переменных в слоях модели на каждом устройстве для всех рабочих процессов. Он использует `CollectiveOps`, операцию TensorFlow для коллективного общения, чтобы агрегировать градиенты и синхронизировать переменные. В [руководстве по `tf.distribute.Strategy`](../../guide/distributed_training.ipynb) есть более подробная информация.

In [ ]:
strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

Примечание: сначала анализируется `TF_CONFIG`, затем, во время вызова `MultiWorkerMirroredStrategy()`, запускаются GRPC сервера TensorFlow. Поэтому переменная окружения `TF_CONFIG` должна быть установлена **до** создания экземпляра `tf.distribute.Strategy`. В нашем случае `TF_CONFIG` еще не установлен и вышеуказанная стратегия фактически представляет собой обучение одного воркера.

`MultiWorkerMirroredStrategy` предоставляет несколько реализаций через [`CollectiveCommunication`](https://github.com/tensorflow/tensorflow/blob/a385a286a930601211d78530734368ccb415bee4/tensorflow/python/distribute/cross_device_ops.parameter). `RING` реализует группы на основе кольца, используя gRPC в качестве уровня связи между хостами. NCCL использует [NCCL Nvidia](https://developer.nvidia.com/nccl) для реализации групп. `AUTO` откладывает выбор на время выполнения. Наилучший выбор реализации групп зависит от количества и типа графических процессоров, а также от сетевого соединения в кластере.

## Тренировка модели

Интеграция API `tf.distribute.Strategy` в `tf.keras` требует единственное изменение для распределния обучения между несколькими воркерами, - это построение модели и вызов `model.compile()` внутри `strategy.scope ()`. Контекст(scope) распределнных стратегий определяет, как и где создаются переменные, а в случае `MultiWorkerMirroredStrategy` создаваемые переменные - это `MirroredVariable`, и они реплицируются на каждый из рабочих процессов.

In [ ]:
with strategy.scope():
  # Создание и компиляция модели должны быть внутри `strategy.scope()`.
  multi_worker_model = mnist.build_and_compile_cnn_model()

Примечание. В настоящее время существует ограничение в `MultiWorkerMirroredStrategy` - операторы TensorFlow должны быть созданы после создания экземпляра стратегии. Если вы видите ошибку `RuntimeError: Collective ops must be configured at program startup`, попробуйте создать экземпляр `MultiWorkerMirroredStrategy` в начале программы и поместите код, который создает операции, после создания стратегии.

Чтобы по-настоящему запустить `MultiWorkerMirroredStrategy`, вам нужно запустить рабочие процессы и передать им `TF_CONFIG`.

Подобно скрипту `mnist.py`, написанному ранее, вот скрипт `main.py`, который будет запускать каждый из воркеров:

In [ ]:
%%writefile main.py

import os
import json

import tensorflow as tf
import mnist

per_worker_batch_size = 64
tf_config = json.loads(os.environ['TF_CONFIG'])
num_workers = len(tf_config['cluster']['worker'])

strategy = tf.distribute.experimental.MultiWorkerMirroredStrategy()

global_batch_size = per_worker_batch_size * num_workers
multi_worker_dataset = mnist.mnist_dataset(global_batch_size)

with strategy.scope():
    multi_worker_model = mnist.build_and_compile_cnn_model()


multi_worker_model.fit(multi_worker_dataset, epochs=3, steps_per_epoch=70)

Обратите внимание, что в приведенном выше коде, для параметра `global_batch_size`, передаваемого в `Dataset.batch`, установлено значение `per_worker_batch_size * num_workers`. Это гарантирует, что каждый рабочий процесс обрабатывает партии размером `per_worker_batch_size`, независимо от количества рабочих.

Текущий каталог теперь содержит оба скрипта:

In [ ]:
%%bash
ls *.py

Теперь нужно сериализовать в json переменную `TF_CONFIG` и добавить ее в переменные окружения:

In [ ]:
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Теперь вы можете запустить рабочий процесс, который будет запускать `main.py` и использовать `TF_CONFIG`:

In [ ]:
# предварительно завершив предыдущий запуск
%killbgscripts

In [ ]:
%%bash --bg
python3 main.py &> job_0.log

Несколько уточнения по команде выше:

1. Команда `%%bash` является ["магией" jupyter notebook](https://ipython.readthedocs.io/en/stable/interactive/magics.html), которая используется для выполнения некоторых команд bash.
2. Флаг `--bg` нужен для запуска процесса `bash` в фоновом режиме, потому что этот рабочий процесс не завершается, а ждет всех воркеров перед запуском.

Фоновый рабочий процесс не будет выводить логи в ноутбук, поэтому `&>` перенаправляет его вывод в файл, чтобы позже вы смогли просмотреть процесс работы.

Подождите несколько секунд, пока процесс запустится:

In [ ]:
import time
time.sleep(10)

Теперь посмотрим, что было выведено в лог-файл воркера:

In [ ]:
%%bash
cat job_0.log

В последней строке лога должно быть написано: `Started server with target: grpc://localhost:12345`. Первый воркер готов и ждет, пока все остальные воркеры будут готовы запуститься.

Обновите `tf_config`, чтобы его подхватил второй рабочий процесс:

In [ ]:
tf_config['task']['index'] = 1
os.environ['TF_CONFIG'] = json.dumps(tf_config)

Теперь запускаем второго воркера. Запуск второго воркера запустит обучение. Так как у нас всего 2 воркеар и они оба активны, - нет необходимости в фоновом процессе:

In [ ]:
%%bash
python3 main.py

Теперь, если вы проверите логи, написанные первым воркером, вы увидите, что он участвовал в обучении модели:

In [ ]:
%%bash
cat job_0.log

Неудивительно, что это медленнее, чем тестовый прогон в начале этого руководства. Запуск нескольких рабочих процессов на одной машине только увеличивает накладные расходы. Целью этого примера было не сократить время, а показать обучение нескольких воркеров.

In [ ]:
# Удалите TF_CONFIG и завершите все фоновые задачи, чтобы они не влияли на следующий раздел.

os.environ.pop('TF_CONFIG', None)
%killbgscripts

## Углубленное обучение нескольких воркеров

До сих пор это мы рассматривали базовые настройки для нескольких рабочих процессов. Остальная часть этого руководства подробно рассматривает другие факторы, которые могут быть полезны для реальных случаев использования.

### Масштабирование данных

При обучении нескольких воркеров необходимо распределение входных данных для обеспечения сходимости и производительности.

В примере из предыдущего раздела используется автомасштабирование по умолчанию, предоставляемое `tf.distribute.Strategy`. Вы можете управлять масштабированием, установив `tf.data.experimental.AutoShardPolicy` для `tf.data.experimental.DistributeOptions`. Чтобы узнать больше об автоматическом масштабировании, см. [Руководство по распределенному вводу данных](https://www.tensorflow.org/tutorials/distribute/input#sharding).

Вот краткий пример того, как отключить автоматическое масштабирование, чтобы каждая реплика обрабатывала каждый пример(не рекомендуется):


In [ ]:
options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.OFF

global_batch_size = 64
multi_worker_dataset = mnist.mnist_dataset(batch_size=64)
dataset_no_auto_shard = multi_worker_dataset.with_options(options)

### Оценка

Если вы передадите `validation_data` в метод `model.fit`, он будет чередовать обучение и оценку для каждой эпохи. Оценка, принимающая `validation_data`, распределяется между одним и тем же набором воркеров, а результаты оценки агрегируются и доступны для всех воркеров. Подобно обучению, набор проверочных данных автоматически разделяется на уровне файла. Вам необходимо установить глобальный размер пакета для проверочного датасета и `validation_steps`. Для оценки также рекомендуется повторяемый набор данных.

В качестве альтернативы вы также можете создать другую независимую задачу, которая будет периодически считывть контрольные точки и запускать оценку. Но это не рекомендуемый способ оценки, поэтому его подробности опускаются.

### Прогноз
Текущая реализация метода `model.predict` не работает с `MultiWorkerMirroredStrategy.`

### Производительность модели

Теперь у вас есть модель Keras, настроеная для работы с несколькими рабочими процессами с помощью `MultiWorkerMirroredStrategy`. Для настройки производительнсоти с помощью `MultiWorkerMirroredStrategy` вы можете сделать следующее:

* `MultiWorkerMirroredStrategy` предоставляет несколько [реализаций коллективного взаимодействия](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/distribute/cross_device_ops.py). `RING` реализует группы на основе кольца, используя gRPC в качестве уровня связи между хостами. `NCCL` использует технологию [NCCL Nvidia](https://developer.nvidia.com/nccl) для реализации групп. `AUTO` откладывает выбор на время выполнения. Наилучший выбор коллективной реализации зависит от количества и типа графических процессоров, а также от сетевого соединения в кластере. Чтобы отменить `AUTO`, укажите одно из значений `RING` или `NCCL`, например `communication = tf.distribute.experimental.CollectiveCommunication.NCCL`.
* Если возможно, преобразуйте переменные в `tf.float`. Официальная модель ResNet включает [пример](https://github.com/tensorflow/models/blob/8367cf6dabe11adf7628541706b660821f397dce/official/resnet/resnet_model.py#L466) того, как это можно сделать.


### Отказоустойчивость

При синхронном обучении весь кластер выйдет из строя, если один из рабочих процессов выйдет из строя, и на данный момент не существует механизма восстановления после сбоя. Использование Keras с `tf.distribute.Strategy` дает некоторую отказоустойчивость в случаях, когда воркеры умирают или работают нестабильно. Вы можете сделать это, сохраняя состояние обучения в распределенной файловой системе по вашему выбору. Таким образом, при перезапуске экземпляра воркера, который ранее завершился некорректно, будет восстановлено предыдущее состояние обучения.

Поскольку все рабочие процессы синхронизированы с точки зрения эпох и шагов обучения, остальным воркерам придется ждать восстановления упавшего воркера, чтобы продолжить общее обучение.

Примечание:
Раньше для восстановления состояния обучения при сбое использовался обратный вызов `ModelCheckpoint`. Теперь команда TensorFlow представляет новый обратный вызов [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w). С этого момента приложения, которые полагаются на `ModelCheckpoint`, должны перейти на новый обратный вызов.

#### Обратный вызов ModelCheckpoint 

Обратный вызов `ModelCheckpoint` больше не обеспечивает поддержку отказоустойчивости, вместо него используйте обратный вызов [`BackupAndRestore`](#scrollTo=kmH8uCUhfn4w).

`ModelCheckpoint` по-прежнему может использоваться для сохранения контрольных точек. Но при этом, если обучение было прервано или успешно завершилось, для продолжения обучения с контрольной точки, пользователь должен загрузить модель вручную.

При желании пользователь может выбрать сохранение и восстановление модели/веса вне функции обратного вызова `ModelCheckpoint`.

<a id='EUNV5Utc1d0s'></a>
### Сохранение и загрузка модели

Чтобы сохранить модель с помощью `model.save` или` tf.saved_model.save`, место сохранения должно быть разным для каждого воркера. На машинах обычных воркеров вам нужно будет сохранить модель во временной директории, а на машине с главным воркером - в предоставленный каталог модели. Временные каталоги на воркере должны быть уникальными, чтобы избежать того, что несколько воркеров пытаются писать в одно и то же место. Модели, сохраненные во всех каталогах, идентичны, и обычно для восстановления или обслуживания следует ссылаться только на модель, сохраненную главным воркером. У вас должна быть некоторая логика очистки, которая удаляет временные каталоги, созданные воркерами, после завершения вашего обучения.

Причина, по которой вам нужно сохранять модель на главном и обычных воркерах одновременно, заключается в том, что вы можете агрегировать переменные во время записи контрольной точки, и для этого требуется, чтобы и руководитель, и воркеры участвовали в протоколе объединения расчетов. С другой стороны, разрешение главному и остальным воркерам сохранять модели в один и тот же каталог приведет к ошибкам записи.

С `MultiWorkerMirroredStrategy` программа запускается для каждого рабочего, и чтобы узнать, является ли текущий работник главным, она использует объект определителя кластера, который имеет атрибуты `task_type` и `task_id`. `task_type` сообщает вам текущее задание (например, `worker_1`), а `task_id` сообщает идентификатор работника, например `1`. Воркер с идентификатором `0` назначается главным.

В приведенном ниже фрагменте кода `write_filepath` возвращает путь к файлу для записи, который зависит от идентификатора воркера. В случае руководителя(работника с идентификатором `0`) он записывает в исходный путь к файлу; для других он создает временный каталог с идентификатором в пути к каталогу:

In [ ]:
model_path = '/tmp/keras-model'

def _is_chief(task_type, task_id):
  # If `task_type` is None, this may be operating as single worker, which works
  # effectively as chief.
  return task_type is None or task_type == 'chief' or (
            task_type == 'worker' and task_id == 0)

def _get_temp_dir(dirpath, task_id):
  base_dirpath = 'workertemp_' + str(task_id)
  temp_dir = os.path.join(dirpath, base_dirpath)
  tf.io.gfile.makedirs(temp_dir)
  return temp_dir

def write_filepath(filepath, task_type, task_id):
  dirpath = os.path.dirname(filepath)
  base = os.path.basename(filepath)
  if not _is_chief(task_type, task_id):
    dirpath = _get_temp_dir(dirpath, task_id)
  return os.path.join(dirpath, base)

task_type, task_id = (strategy.cluster_resolver.task_type,
                      strategy.cluster_resolver.task_id)
write_model_path = write_filepath(model_path, task_type, task_id)

Теперь вы готовы сохранить:

In [ ]:
multi_worker_model.save(write_model_path)

Как описано выше, модель загружается только из пути, в который сохранял главный воркер, поэтому давайте удалим временные директории, которые сохранили остальные воркеры:

In [ ]:
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(os.path.dirname(write_model_path))

Теперь, когда нужно загрузить модель, воспользуемся API `tf.keras.models.load_model` и продолжим обучение. Предположим, что для загрузки и продолжения обучения используется только один воркер, в таком случае вам не нужно вызывать `tf.keras.models.load_model` внутри `strategy.scope()`.

In [ ]:
loaded_model = tf.keras.models.load_model(model_path)

# Модель восстановлена и мы можем продолжить обучение.
loaded_model.fit(single_worker_dataset, epochs=2, steps_per_epoch=20)

### Сохранение и восстановление контрольной точки(чекпойнта)

Контрольные точки позволяют сохранять веса модели и восстанавливать их без необходимости сохранять и восстанавливать всю модель. Далее вы создадите один `tf.train.Checkpoint` для отслеживания модели, которая управляется `tf.train.CheckpointManager` и сохраняет только последний чекпойнт.

In [ ]:
checkpoint_dir = '/tmp/ckpt'

checkpoint = tf.train.Checkpoint(model=multi_worker_model)
write_checkpoint_dir = write_filepath(checkpoint_dir, task_type, task_id)
checkpoint_manager = tf.train.CheckpointManager(
    checkpoint, directory=write_checkpoint_dir, max_to_keep=1)

После создания `CheckpointManager` вы сохраняете чекпойнты, а затем удалите все чекпойнты, созданные обычными воркерами(не главным).

In [ ]:
checkpoint_manager.save()
if not _is_chief(task_type, task_id):
  tf.io.gfile.rmtree(write_checkpoint_dir)

Теперь, когда вам нужно восстановить чекпойнт, вы можете найти последний с помощью удобной функции `tf.train.latest_checkpoint`. После восстановления чекпойнта можно продолжить обучение.

In [ ]:
latest_checkpoint = tf.train.latest_checkpoint(checkpoint_dir)
checkpoint.restore(latest_checkpoint)
multi_worker_model.fit(multi_worker_dataset, epochs=2, steps_per_epoch=20)

#### Обратный вызов BackupAndRestore

Примечание: обратный вызов `tf.keras.callbacks.experimental.BackupAndRestore` доступен только в ночных сборках tf-nightly.

Обратный вызов `BackupAndRestore` обеспечивает функциональность отказоустойчивости, создавая резервную копию модели и текущего номера эпохи во временном файле, переданном в аргументе `backup_dir` для `BackupAndRestore`. Это делается в конце каждой эпохи.

Если задания прерываются и перезапускаются, `BackupAndRestore` восстанавливает последнюю контрольную точку, и обучение продолжается с начала прерванной эпохи. Любое частичное обучение, уже выполненное в незавершенную эпоху до прерывания, будет отброшено, и не повлияет на окончательное состояние модели.

Чтобы `BackupAndRestore`, передайте его экземпляр при вызове `tf.keras.Model.fit()`.

При использовании `MultiWorkerMirroredStrategy`, если воркер прерывается, весь кластер приостанавливается, пока прерванный воркер не будет перезапущен. Другие воркеры также перезапустятся, и прерванный воркер снова присоединится к кластеру. Затем каждый воркер читает файл контрольной точки, который был ранее сохранен, и загружает его прежнее состояние, тем самым позволяя кластеру синхронизироваться. Затем обучение продолжается.

Обратный вызов `BackupAndRestore` для сохранения и восстановления состояния обучения использует `CheckpointManager`, который, в свою очередь генерирует файл, называемый контрольной точкой, и отслеживает существующие контрольные точки вместе с последней. По этой причине не следует повторно использовать одно и тоже значение `backup_dir`, чтобы избежать конфликта имен.

В настоящее время обратный вызов `BackupAndRestore` поддерживает одного воркера без стратегии, одного воркера с `MirroredStrategy`, и нескольких воркеров с помощью `MultiWorkerMirroredStrategy`.
Ниже приведены два примера обучения нескольких воркеров и обучение одного воркера.

In [ ]:
# Multi-worker training with MultiWorkerMirroredStrategy.

callbacks = [tf.keras.callbacks.experimental.BackupAndRestore(backup_dir='/tmp/backup')]
with strategy.scope():
  multi_worker_model = mnist.build_and_compile_cnn_model()
multi_worker_model.fit(multi_worker_dataset,
                       epochs=3,
                       steps_per_epoch=70,
                       callbacks=callbacks)

Если вы проверите каталог `backup_dir`, который вы указали в `BackupAndRestore`, вы увидите временно сгенерированные файлы контрольных точек. Эти файлы нужны для восстановления ранее утерянных экземпляров, и они будут удалены библиотекой в конце `tf.keras.Model.fit()`, после успешного завершения вашего обучения.

Примечание. В настоящее время `BackupAndRestore` поддерживает только активный режим. В графическом режиме рассмотрите возможность использования [Сохранение и загрузка модели](#scrollTo=EUNV5Utc1d0s), упомянутого выше.

## Смотрите также
1. В руководстве [Распределенное обучение в TensorFlow](https://www.tensorflow.org/guide/distributed_training) представлен обзор доступных стратегий распределения.
2. [Официальные модели](https://github.com/tensorflow/models/tree/master/official), многие из которых можно настроить для запуска с несколькими стратегиями.
3. Раздел [Производительность](../../guide/function.ipynb) предоставляет информацию о других стратегиях и [инструментах](../../guide/profiler.md), которые можно использовать для оптимизации производительности ваших моделей TensorFlow.
